In [1]:
from google.colab import files
uploaded = files.upload()


Saving sorular.csv to sorular.csv


In [4]:
import pandas as pd

df = pd.read_csv(list(uploaded.keys())[0])
df.head()

,ID,Soru_Metni(OCR),Alt_Konu,Zorluk,Görsel_Tipi,Kaynak_Tipi
0,P2-S1,18 sayısını bölen pozitif tam sayıların toplam...,Bir Doğal Sayının Çarpanları,1,Görsel Yok,Başlangıç Sorusu
1,P2-S2,Aşağıdakilerden hangisi 96 sayısının çarpanlar...,Bir Doğal Sayının Çarpanları,1,Görsel Yok,Başlangıç Sorusu
2,P2-S3,Aşağıdakilerden hangisi bu dikdörtgensel bölge...,Bir Doğal Sayının Çarpanları,2,Görsel Yok,Başlangıç Sorusu
3,P2-S4,Bu dikdörtgenlerden çevresi en küçük olanın çe...,Bir Doğal Sayının Çarpanları,3,Görsel Yok,Başlangıç Sorusu
4,P2-S5,Misafirlerin sayısı 6'dan fazla olduğuna göre ...,Bir Doğal Sayının Çarpanları,2,Görsel Yok,Başlangıç Sorusu


In [5]:
df.columns

Index(['ID', 'Soru_Metni(OCR)', 'Alt_Konu', 'Zorluk', 'Görsel_Tipi',
       'Kaynak_Tipi'],
      dtype='object')

In [6]:
def clean_column_name(col):
    return (col
            .strip()
            .replace(" ", "_")
            .replace("-", "_")
            .replace("(", "")
            .replace(")", "")
            .replace("Ö", "O")
            .replace("ö", "o")
            .replace("Ü", "U")
            .replace("ü", "u")
            .replace("Ş", "S")
            .replace("ş", "s")
            .replace("Ğ", "G")
            .replace("ğ", "g")
            .replace("İ", "I")
            .replace("ı", "i")
           )
df.columns = [clean_column_name(c) for c in df.columns]
df.columns


Index(['ID', 'Soru_MetniOCR', 'Alt_Konu', 'Zorluk', 'Gorsel_Tipi',
       'Kaynak_Tipi'],
      dtype='object')

In [7]:
df['Zorluk'] = df['Zorluk'].astype(int)

In [8]:
df['Zorluk'] = df['Zorluk'].astype(str).str.strip().astype(int)

In [9]:
df['Gorsel_Tipi'].unique()

array(['Görsel Yok', 'Resimli', 'Geometrik Şekil', 'Şematik', 'Yok',
       'Tablo'], dtype=object)

In [11]:
# Normalize Görsel Tipi
def normalize_gorsel(x):
    x = str(x).strip()
    mapping = {
        "Görsel Yok": "yok",
        "Yok": "yok",
        "Resimli": "resimli",
        "Geometrik Şekil": "geometrik_sekil",
        "Şematik": "sematik",
        "Tablo": "tablo",
    }
    return mapping.get(x, x.lower().replace(" ", "_"))

df["Gorsel_Tipi"] = df["Gorsel_Tipi"].apply(normalize_gorsel)

df["Gorsel_Tipi"].unique()


array(['yok', 'resimli', 'geometrik_sekil', 'sematik', 'tablo'],
      dtype=object)

In [10]:
df['Kaynak_Tipi'].unique()

array(['Başlangıç Sorusu', 'LGS Örnek Sorusu', 'LGS Örnek Soru',
       'Çıkmış Soru (2025)', 'Çıkmış Soru (2024)', 'Çıkmış Soru (2023)',
       'Çıkmış Soru (2022)', 'Çıkmış Soru (2021)', 'Çıkmış Soru (2020)',
       'Çıkmış Soru (2019)', 'Çıkmış Soru (2018)'], dtype=object)

In [12]:
def normalize_kaynak(x):
    x = str(x).strip()

    if "Başlangıç" in x:
        return "baslangic"

    if "Örnek" in x or "Ornek" in x:
        return "ornek"

    if "Çıkmış" in x or "Cikmis" in x:
        return "cikmis"

    return x.lower()

df["Kaynak_Tipi"] = df["Kaynak_Tipi"].apply(normalize_kaynak)
df["Kaynak_Tipi"].unique()


array(['baslangic', 'ornek', 'cikmis'], dtype=object)

In [13]:
df['Alt_Konu'].unique()

array(['Bir Doğal Sayının Çarpanları', 'EBOB ve EKOK',
       'Aralarında Asal Sayılar'], dtype=object)

In [14]:
df['Alt_Konu'] = df['Alt_Konu'].str.strip()

In [15]:
df['Alt_Konu'].unique()

array(['Bir Doğal Sayının Çarpanları', 'EBOB ve EKOK',
       'Aralarında Asal Sayılar'], dtype=object)

In [16]:
alt_konu_map = {
    "Bir Doğal Sayının Çarpanları": "carpanlar",
    "EBOB ve EKOK": "ebob_ekok",
    "Aralarında Asal Sayılar": "aralarinda_asal",
}

df["Alt_Konu"] = df["Alt_Konu"].map(alt_konu_map)
df["Alt_Konu"].unique()


array(['carpanlar', 'ebob_ekok', 'aralarinda_asal'], dtype=object)

In [17]:
def get_weight(x):
    return {
        "cikmis": 1.0,
        "ornek": 0.8,
        "baslangic": 0.3
    }.get(x, 1.0)

df["sample_weight"] = df["Kaynak_Tipi"].apply(get_weight)

In [18]:
df.head()

,ID,Soru_MetniOCR,Alt_Konu,Zorluk,Gorsel_Tipi,Kaynak_Tipi,sample_weight
0,P2-S1,18 sayısını bölen pozitif tam sayıların toplam...,carpanlar,1,yok,baslangic,0.3
1,P2-S2,Aşağıdakilerden hangisi 96 sayısının çarpanlar...,carpanlar,1,yok,baslangic,0.3
2,P2-S3,Aşağıdakilerden hangisi bu dikdörtgensel bölge...,carpanlar,2,yok,baslangic,0.3
3,P2-S4,Bu dikdörtgenlerden çevresi en küçük olanın çe...,carpanlar,3,yok,baslangic,0.3
4,P2-S5,Misafirlerin sayısı 6'dan fazla olduğuna göre ...,carpanlar,2,yok,baslangic,0.3


In [19]:
df = df.drop(columns=["ID"])

In [20]:
df.to_csv("cleaned_dataset.csv", index=False)

In [21]:
df["is_LGS"] = (df["Kaynak_Tipi"].isin(["ornek", "cikmis"])).astype(int)

In [22]:
# Kolon adı
col = "Soru_MetniOCR"

# 1) Kelime sayisi
df["ocr_kelime_sayisi"] = df[col].str.split().str.len()

# 2) Karakter sayisi
df["ocr_karakter_sayisi"] = df[col].str.len()

# 3) Rakam sayisi
df["ocr_rakam_sayisi"] = df[col].str.count(r"\d")

# 4) Cok adimli muhakeme ifadesi
reasoning_words = [
    "en az", "en cok", "en fazla",
    "buna gore", "once", "daha sonra",
    "en son", "ardindan", "son durumda"
]

pattern_reason = "|".join(reasoning_words)
df["ocr_cok_adimli"] = df[col].str.contains(
    pattern_reason, case=False, regex=True
).astype(int)


In [23]:
df = df.rename(columns={"sample_weight": "egitim_agirligi"})

In [24]:
df.to_csv("dataset_ocr_li.csv", index=False, encoding="utf-8")


In [25]:
df_no_ocr = df.drop(columns=["Soru_MetniOCR"])


In [26]:
df_no_ocr.to_csv("dataset_model_final.csv", index=False, encoding="utf-8")
